In [0]:
# !pip uninstall tensorflow
# !pip install tensorflow==2.0.0

In [0]:
%matplotlib inline
%load_ext tensorboard
# %tensorboard --logdir logs

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.transform import resize

import tensorflow as tf

from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dense, Dropout

In [3]:
tf.__version__

'2.0.0'

# Neural Network Architectures

In [0]:
model = VGG19()

574717952/574710816 [==============================] - 17s 0us/step


In [0]:
model.summary()

In [0]:
cat_image = imread("https://a57.foxnews.com/static.foxnews.com/foxnews.com/content/uploads/2019/09/1862/1048/cat-getty-images.jpg?ve=1&tl=1")

In [0]:
cat_image.shape

(1048, 1862, 3)

In [0]:
preprocess_input(cat_image).max()

151.061

In [0]:
cat_img_resized = resize(cat_image, (224,224))

In [0]:
cat_img_resized.max()

0.9921568627450981

In [0]:
cat_img_preprocessed = preprocess_input(cat_img_resized * 255)
cat_img_preproc_batch = np.array([cat_img_preprocessed])

In [0]:
cat_img_preprocessed.max()

149.061

In [0]:
cat_img_preproc_batch.shape

(1, 224, 224, 3)

In [0]:
predictions = model.predict(cat_img_preproc_batch)

In [0]:
decode_predictions(predictions)

40960/35363 [==================================] - 0s 0us/step


[[('n02124075', 'Egyptian_cat', 0.31052068),
  ('n02127052', 'lynx', 0.09960841),
  ('n02123159', 'tiger_cat', 0.07274307),
  ('n02123045', 'tabby', 0.038776107),
  ('n02123597', 'Siamese_cat', 0.027249739)]]

## Transfer Learning

In [4]:
vgg16 = VGG16()

553467904/553467096 [==============================] - 43s 0us/step


In [5]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
input_layer = vgg16.get_layer("input_1")
output_layer = vgg16.get_layer("flatten")
custom_model = Model(inputs=input_layer.input, outputs=output_layer.output)
custom_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
custom_model.trainable = False
custom_model_full = Sequential([
                                custom_model,
                                Dense(256, activation="relu"),
                                Dropout(0.1),
                                Dense(1, activation="sigmoid")
])
custom_model_full.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 25088)             14714688  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 21,137,729
Trainable params: 6,423,041
Non-trainable params: 14,714,688
_________________________________________________________________


## Adversarial Networks

In [0]:
# input_image = np.zeros((224,224,3))
input_image = np.random.uniform(0, 255, (224,224,3)).astype(np.uint8)


In [0]:
input_image

In [0]:
input_image_preprocess = preprocess_input(input_image)

In [0]:
predictions = vgg16.predict([[input_image_preprocess]])

In [0]:
decoded_predictions = decode_predictions(predictions)

In [21]:
decoded_predictions

[[('n03223299', 'doormat', 0.14682753),
  ('n03998194', 'prayer_rug', 0.11977561),
  ('n04589890', 'window_screen', 0.07550588),
  ('n04525038', 'velvet', 0.05493306),
  ('n03980874', 'poncho', 0.050309513)]]